In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.metrics import f1_score
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.feature_extraction.text import TfidfVectorizer
import tqdm, sys, os, gc, re, argparse, warnings
warnings.filterwarnings('ignore')



In [2]:
train = pd.read_excel('./dataset-new/traindata-new.xlsx')
test = pd.read_excel('./dataset-new/testdata-new.xlsx')

# test数据不包含 DC50 (nM) 和 Dmax (%)
train = train.drop(['DC50 (nM)', 'Dmax (%)'], axis=1)

# 定义了一个空列表drop_cols，用于存储在测试数据集中非空值小于10个的列名。
drop_cols = []
for f in test.columns:
    if test[f].notnull().sum() < 10:
        drop_cols.append(f)
        
# 使用drop方法从训练集和测试集中删除了这些列，以避免在后续的分析或建模中使用这些包含大量缺失值的列
train = train.drop(drop_cols, axis=1)
test = test.drop(drop_cols, axis=1)

# 使用pd.concat将清洗后的训练集和测试集合并成一个名为data的DataFrame，便于进行统一的特征工程处理
data = pd.concat([train, test], axis=0, ignore_index=True)
cols = data.columns[2:]


In [3]:
# 将SMILES转换为分子对象列表,并转换为SMILES字符串列表
data['smiles_list'] = data['Smiles'].apply(lambda x:[Chem.MolToSmiles(mol, isomericSmiles=True) for mol in [Chem.MolFromSmiles(x)]])
data['smiles_list'] = data['smiles_list'].map(lambda x: ' '.join(x))  

# 使用TfidfVectorizer计算TF-IDF
tfidf = TfidfVectorizer(max_df = 0.9, min_df = 1, sublinear_tf = True)
res = tfidf.fit_transform(data['smiles_list'])

# 将结果转为dataframe格式
tfidf_df = pd.DataFrame(res.toarray())
tfidf_df.columns = [f'smiles_tfidf_{i}' for i in range(tfidf_df.shape[1])]

# 按列合并到data数据
data = pd.concat([data, tfidf_df], axis=1)

# 自然数编码
def label_encode(series):
    unique = list(series.unique())
    return series.map(dict(zip(
        unique, range(series.nunique())
    )))

for col in cols:
    if data[col].dtype == 'object':
        data[col]  = label_encode(data[col])
        
train = data[data.Label.notnull()].reset_index(drop=True)
test = data[data.Label.isnull()].reset_index(drop=True)

# 特征筛选
features = [f for f in train.columns if f not in ['uuid','Label','smiles_list']]

# 构建训练集和测试集
x_train = train[features]
x_test = test[features]

# 训练集标签
y_train = train['Label'].astype(int)


In [4]:
def cv_model(clf, train_x, train_y, test_x, clf_name, seed=2022):
    
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)

    train = np.zeros(train_x.shape[0])
    test = np.zeros(test_x.shape[0])

    cv_scores = []
    # 100， 1 2 3 4 5
    # 1 2 3 4    5
    # 1 2 3 5。  4
    # 1
    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} {}************************************'.format(str(i+1), str(seed)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]
               
        params = {'learning_rate': 0.1, 'depth': 6, 'l2_leaf_reg': 10, 'bootstrap_type':'Bernoulli','random_seed':seed,
                  'od_type': 'Iter', 'od_wait': 100, 'allow_writing_files': False, 'task_type':'CPU'}

        model = clf(iterations=20000, **params, eval_metric='AUC')
        model.fit(trn_x, trn_y, eval_set=(val_x, val_y),
                  metric_period=100,
                  cat_features=[], 
                  use_best_model=True, 
                  verbose=1)

        val_pred  = model.predict_proba(val_x)[:,1]
        test_pred = model.predict_proba(test_x)[:,1]
            
        train[valid_index] = val_pred
        test += test_pred / kf.n_splits
        cv_scores.append(f1_score(val_y, np.where(val_pred>0.5, 1, 0)))
        
        print(cv_scores)
       
    print("%s_score_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    return train, test
    
cat_train, cat_test = cv_model(CatBoostClassifier, x_train, y_train, x_test, "cat")

pd.DataFrame(
    {
        'uuid': test['uuid'],
        'Label': np.where(cat_test>0.5, 1, 0)
    }
).to_csv('submit.csv', index=None)


************************************ 1 2022************************************
0:	test: 0.7979798	best: 0.7979798 (0)	total: 135ms	remaining: 45m 9s


100:	test: 0.9048822	best: 0.9149832 (82)	total: 345ms	remaining: 1m 7s
200:	test: 0.9158249	best: 0.9200337 (173)	total: 537ms	remaining: 52.8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.92003367
bestIteration = 173

Shrink model to first 174 iterations.
[0.9032258064516128]
************************************ 2 2022************************************
0:	test: 0.7527174	best: 0.7527174 (0)	total: 2.84ms	remaining: 56.9s


100:	test: 0.9211957	best: 0.9248188 (98)	total: 207ms	remaining: 40.8s
200:	test: 0.9393116	best: 0.9411232 (197)	total: 410ms	remaining: 40.4s
300:	test: 0.9465580	best: 0.9465580 (296)	total: 620ms	remaining: 40.6s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.946557971
bestIteration = 296

Shrink model to first 297 iterations.
[0.9032258064516128, 0.9166666666666666]
************************************ 3 2022************************************
0:	test: 0.7349676	best: 0.7349676 (0)	total: 3.05ms	remaining: 1m 1s


100:	test: 0.8977798	best: 0.9024052 (44)	total: 200ms	remaining: 39.5s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9024051804
bestIteration = 44

Shrink model to first 45 iterations.
[0.9032258064516128, 0.9166666666666666, 0.8823529411764706]
************************************ 4 2022************************************
0:	test: 0.7991837	best: 0.7991837 (0)	total: 4.13ms	remaining: 1m 22s


100:	test: 0.9040816	best: 0.9155102 (21)	total: 205ms	remaining: 40.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9155102041
bestIteration = 21

Shrink model to first 22 iterations.
[0.9032258064516128, 0.9166666666666666, 0.8823529411764706, 0.8333333333333333]
************************************ 5 2022************************************
0:	test: 0.8192935	best: 0.8192935 (0)	total: 3.35ms	remaining: 1m 6s


100:	test: 0.9492754	best: 0.9682971 (43)	total: 207ms	remaining: 40.8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9682971014
bestIteration = 43

Shrink model to first 44 iterations.
[0.9032258064516128, 0.9166666666666666, 0.8823529411764706, 0.8333333333333333, 0.9072164948453608]
cat_score_list: [0.9032258064516128, 0.9166666666666666, 0.8823529411764706, 0.8333333333333333, 0.9072164948453608]
cat_score_mean: 0.8885590484946888
cat_score_std: 0.02980485757632575
